# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fd5744ad850>
├── 'a' --> tensor([[ 0.4811,  0.7283, -0.9325],
│                   [ 0.5699,  0.8484, -0.7635]])
└── 'x' --> <FastTreeValue 0x7fd5744adf10>
    └── 'c' --> tensor([[-0.5616,  0.0933, -0.9277,  0.2709],
                        [-0.6223,  1.1157,  0.8409,  0.3695],
                        [ 0.5501, -2.5902, -0.3695,  1.9894]])

In [4]:
t.a

tensor([[ 0.4811,  0.7283, -0.9325],
        [ 0.5699,  0.8484, -0.7635]])

In [5]:
%timeit t.a

87.7 ns ± 0.334 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fd5744ad850>
├── 'a' --> tensor([[ 0.1555, -0.0068,  0.1731],
│                   [ 0.0522, -2.4731,  0.7801]])
└── 'x' --> <FastTreeValue 0x7fd5744adf10>
    └── 'c' --> tensor([[-0.5616,  0.0933, -0.9277,  0.2709],
                        [-0.6223,  1.1157,  0.8409,  0.3695],
                        [ 0.5501, -2.5902, -0.3695,  1.9894]])

In [7]:
%timeit t.a = new_value

91.6 ns ± 0.308 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.4811,  0.7283, -0.9325],
               [ 0.5699,  0.8484, -0.7635]]),
    x: Batch(
           c: tensor([[-0.5616,  0.0933, -0.9277,  0.2709],
                      [-0.6223,  1.1157,  0.8409,  0.3695],
                      [ 0.5501, -2.5902, -0.3695,  1.9894]]),
       ),
)

In [10]:
b.a

tensor([[ 0.4811,  0.7283, -0.9325],
        [ 0.5699,  0.8484, -0.7635]])

In [11]:
%timeit b.a

77.2 ns ± 0.49 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 1.4246, -1.1421,  0.8382],
               [-0.1473,  1.2942, -0.5401]]),
    x: Batch(
           c: tensor([[-0.5616,  0.0933, -0.9277,  0.2709],
                      [-0.6223,  1.1157,  0.8409,  0.3695],
                      [ 0.5501, -2.5902, -0.3695,  1.9894]]),
       ),
)

In [13]:
%timeit b.a = new_value

637 ns ± 3.97 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

1.05 µs ± 8.77 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

12.1 µs ± 72.4 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

170 µs ± 1.93 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

171 µs ± 1.79 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fd49fb32460>
├── 'a' --> tensor([[[ 0.4811,  0.7283, -0.9325],
│                    [ 0.5699,  0.8484, -0.7635]],
│           
│                   [[ 0.4811,  0.7283, -0.9325],
│                    [ 0.5699,  0.8484, -0.7635]],
│           
│                   [[ 0.4811,  0.7283, -0.9325],
│                    [ 0.5699,  0.8484, -0.7635]],
│           
│                   [[ 0.4811,  0.7283, -0.9325],
│                    [ 0.5699,  0.8484, -0.7635]],
│           
│                   [[ 0.4811,  0.7283, -0.9325],
│                    [ 0.5699,  0.8484, -0.7635]],
│           
│                   [[ 0.4811,  0.7283, -0.9325],
│                    [ 0.5699,  0.8484, -0.7635]],
│           
│                   [[ 0.4811,  0.7283, -0.9325],
│                    [ 0.5699,  0.8484, -0.7635]],
│           
│                   [[ 0.4811,  0.7283, -0.9325],
│                    [ 0.5699,  0.8484, -0.7635]]])
└── 'x' --> <FastTreeValue 0x7fd49fb32280>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

38.9 µs ± 687 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fd4964c2f40>
├── 'a' --> tensor([[ 0.4811,  0.7283, -0.9325],
│                   [ 0.5699,  0.8484, -0.7635],
│                   [ 0.4811,  0.7283, -0.9325],
│                   [ 0.5699,  0.8484, -0.7635],
│                   [ 0.4811,  0.7283, -0.9325],
│                   [ 0.5699,  0.8484, -0.7635],
│                   [ 0.4811,  0.7283, -0.9325],
│                   [ 0.5699,  0.8484, -0.7635],
│                   [ 0.4811,  0.7283, -0.9325],
│                   [ 0.5699,  0.8484, -0.7635],
│                   [ 0.4811,  0.7283, -0.9325],
│                   [ 0.5699,  0.8484, -0.7635],
│                   [ 0.4811,  0.7283, -0.9325],
│                   [ 0.5699,  0.8484, -0.7635],
│                   [ 0.4811,  0.7283, -0.9325],
│                   [ 0.5699,  0.8484, -0.7635]])
└── 'x' --> <FastTreeValue 0x7fd49752b130>
    └── 'c' --> tensor([[-0.5616,  0.0933, -0.9277,  0.2709],
                        [-0.6223,  1.1157,  0.8409,  0.3695],
                 

In [23]:
%timeit t_cat(trees)

35.9 µs ± 222 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

73.5 µs ± 722 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 0.4811,  0.7283, -0.9325],
                [ 0.5699,  0.8484, -0.7635]],
       
               [[ 0.4811,  0.7283, -0.9325],
                [ 0.5699,  0.8484, -0.7635]],
       
               [[ 0.4811,  0.7283, -0.9325],
                [ 0.5699,  0.8484, -0.7635]],
       
               [[ 0.4811,  0.7283, -0.9325],
                [ 0.5699,  0.8484, -0.7635]],
       
               [[ 0.4811,  0.7283, -0.9325],
                [ 0.5699,  0.8484, -0.7635]],
       
               [[ 0.4811,  0.7283, -0.9325],
                [ 0.5699,  0.8484, -0.7635]],
       
               [[ 0.4811,  0.7283, -0.9325],
                [ 0.5699,  0.8484, -0.7635]],
       
               [[ 0.4811,  0.7283, -0.9325],
                [ 0.5699,  0.8484, -0.7635]]]),
    x: Batch(
           c: tensor([[[-0.5616,  0.0933, -0.9277,  0.2709],
                       [-0.6223,  1.1157,  0.8409,  0.3695],
                       [ 0.5501, -2.5902, -0.3695,  1.9894]],
         

In [26]:
%timeit Batch.stack(batches)

98.4 µs ± 369 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 0.4811,  0.7283, -0.9325],
               [ 0.5699,  0.8484, -0.7635],
               [ 0.4811,  0.7283, -0.9325],
               [ 0.5699,  0.8484, -0.7635],
               [ 0.4811,  0.7283, -0.9325],
               [ 0.5699,  0.8484, -0.7635],
               [ 0.4811,  0.7283, -0.9325],
               [ 0.5699,  0.8484, -0.7635],
               [ 0.4811,  0.7283, -0.9325],
               [ 0.5699,  0.8484, -0.7635],
               [ 0.4811,  0.7283, -0.9325],
               [ 0.5699,  0.8484, -0.7635],
               [ 0.4811,  0.7283, -0.9325],
               [ 0.5699,  0.8484, -0.7635],
               [ 0.4811,  0.7283, -0.9325],
               [ 0.5699,  0.8484, -0.7635]]),
    x: Batch(
           c: tensor([[-0.5616,  0.0933, -0.9277,  0.2709],
                      [-0.6223,  1.1157,  0.8409,  0.3695],
                      [ 0.5501, -2.5902, -0.3695,  1.9894],
                      [-0.5616,  0.0933, -0.9277,  0.2709],
                      [-0.6223,  

In [28]:
%timeit Batch.cat(batches)

180 µs ± 576 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

454 µs ± 31.4 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
